In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
os.listdir() 

['.DS_Store',
 'deap dataset.ipynb',
 'Emotion analysis implementation.ipynb',
 'EmotionsData.csv',
 '.ipynb_checkpoints',
 'HCI-Emotion-analysis',
 'data_preprocessed_python',
 'FinalScript.ipynb']

1. We will take into consideration only one video 
2. Loop over all files 
3. for each file --> readFile
4. Convert to dataframe
5. Extract only data for the first element from data 
6. Extract labels data
6. Make average data 
7. Put in data dataFrame

8. Extract data as csv

In [3]:
# public general variables
fileNamesArr = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', ]

all_channels = np.array(['b1','b2', 'b3', 'b4', 'b5', 'b6','b7', 'b8', 'b9', 'b10','b11','b12', 'b13', 'b14', 'b15', 'b16','b17', 'b18', 'b19', 'b20','b21','b22', 'b23', 'b24', 'b25', 'b26','b27', 'b28', 'b29', 'b30', 'b31', 'b32', 'hEOG', 'vEOG', 'zEMG', 'tEMG', 'GSR', 'Respiration belt', 'Plethysmograph', 'Temp'])
EEG_data = np.array(['b1','b2', 'b3', 'b4', 'b5', 'b6','b7', 'b8', 'b9', 'b10','b11','b12', 'b13', 'b14', 'b15', 'b16','b17', 'b18', 'b19', 'b20','b21','b22', 'b23', 'b24', 'b25', 'b26','b27', 'b28', 'b29', 'b30', 'b31', 'b32'])
peripheral_channels = np.array(["zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

finalData = pd.DataFrame(columns = ["subjectID", "valence", "arousal", "dominance", "VA", "VD", "AD", "VAD", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])


In [9]:
# Reading the files where the data is in shape of 40 * 40 * 8064

def readFile(fileNumber):
    infile = open('data_preprocessed_python/s' + fileNumber + '.dat', 'rb')
    new_dict = pickle.load(infile, encoding="latin1")
    infile.close()
    labels=[]
    data=[]

    for keys in new_dict:
        if(keys=='labels'):
            labels=new_dict[keys]
        else:
            data=new_dict[keys]
            
    return labels, data

#####################################################################################################################
def convertToDataframe(data):
    # drop EEG data 
    data = data[34:, :]
    
    df_data = pd.DataFrame(columns = peripheral_channels)
    df_dataPerVideo = pd.DataFrame()

    for i in range(0, len(data)): #video 40
        signal = data[i]
        df_dataPerVideo[peripheral_channels[i]] = signal
    
    return df_dataPerVideo

def averageData(data):
    averageData = []
    for i in range(0, len(peripheral_channels)):
        averageData.append(data[peripheral_channels[i]].mean())
    return averageData

#####################################################################################################################
def getMedians(labels_separated):
    valenceMedian = labels_separated['valence'].median()
    arousalMedian = labels_separated['arousal'].median()
    dominanceMedian = labels_separated['dominance'].median()
    
    return valenceMedian, arousalMedian, dominanceMedian

#####################################################################################################################
def getLabels(labels):
    valence = labelsArr[:,0]
    arousal = labelsArr[:,1]
    dominance = labelsArr[:,2]
    liking = labelsArr[:,3]

    labels_separated = pd.DataFrame({'valence': valence, 'arousal': arousal, 'dominance': dominance, 'liking': liking })

    valenceMedian, arousalMedian, dominanceMedian = getMedians(labels_separated)
    
    labelledData = []
#     labelledData.append("pos" if labels_separated.iloc[0]['valence'] >= valenceMedian else "neg")
#     labelledData.append("high" if labels_separated.iloc[0]['arousal'] >= arousalMedian else "low")
#     labelledData.append("strong" if labels_separated.iloc[0]['dominance'] >= dominanceMedian else "weak")
    labelledData.append(1 if labels_separated.iloc[0]['valence'] >= valenceMedian else 0)
    labelledData.append(1 if labels_separated.iloc[0]['arousal'] >= arousalMedian else 0)
    labelledData.append(1 if labels_separated.iloc[0]['dominance'] >= dominanceMedian else 0)
    
    return labelledData

#####################################################################################################################
def processLabels(labels):
    valence = labelsArr[:,0]
    arousal = labelsArr[:,1]
    dominance = labelsArr[:,2]
    liking = labelsArr[:,3]

    labels_separated = pd.DataFrame({'valence': valence, 'arousal': arousal, 'dominance': dominance, 'liking': liking })

    return labels_separated    

#####################################################################################################################
def concatAllData(fileNames, labelsEncoded, avgData):
    df_fileNames = pd.DataFrame(columns = ["subjectID"])
    df_avgData = pd.DataFrame(columns = peripheral_channels)
    df_labelsEncoded = pd.DataFrame(columns = ["valence", "arousal", "dominance"])
    concatedData = pd.DataFrame(columns = ["subjectID", "valence", "arousal", "dominance", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

    df_length = len(df_fileNames)
    df_fileNames.loc[df_length] = fileNumber

    df_length = len(df_labelsEncoded)
    df_labelsEncoded.loc[df_length] = labelsEncoded

    df_length = len(df_avgData)
    df_avgData.loc[df_length] = avgData

    concatedData = pd.concat([df_fileNames, df_labelsEncoded, df_avgData], axis=1)
    return concatedData

#####################################################################################################################
def computeSum(finalData):
    finalData['VA'] = finalData['valence'] + finalData['arousal']
    finalData['VD'] = finalData['valence'] + finalData['dominance']
    finalData['AD'] = finalData['arousal'] + finalData['dominance']
    finalData['VAD'] = finalData['valence'] + finalData['arousal'] + finalData['dominance']

    return finalData


In [10]:
finalData = pd.DataFrame(columns = ["subjectID", "valence", "arousal", "dominance", "VA", "VD", "AD", "VAD", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

for i in range(0, len(fileNamesArr)):
#     print(i)
    fileNumber = fileNamesArr[i]
    labelsArr, dataArr = readFile(fileNumber)
    
    for j in range(0, len(dataArr[i])):
        data_df = convertToDataframe(dataArr[j]) #only data for the first video
        avgData = averageData(data_df)
        df_labelsEncoded = getLabels(labelsArr)

        concatedData = concatAllData(fileNamesArr, df_labelsEncoded, avgData)
        finalData = pd.concat([finalData, concatedData], ignore_index=True,)
        finalData = computeSum(finalData)
finalData

,subjectID,valence,arousal,dominance,VA,VD,AD,VAD,zEMG,tEMG,GSR,Respiration belt,Plethysmograph,Temperature
0,01,1,1,1,2,2,2,3,19.525680,-15.428396,1716.504173,441.855164,-209.929106,-0.042336
1,01,1,1,1,2,2,2,3,1.304476,-182.562756,-4327.190364,990.629959,4484.697933,-0.016863
2,01,1,1,1,2,2,2,3,-15.389395,-112.106246,-43738.214008,597.245885,10739.267212,-0.029897
3,01,1,1,1,2,2,2,3,240.601412,41.611802,9473.421272,-31.355322,-2767.565063,-0.016429
4,01,1,1,1,2,2,2,3,-7.129091,-221.357590,5486.095310,-597.137189,-12165.254217,-0.041215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,32,1,0,1,1,2,1,2,72.804410,-221.680595,37.132492,52.180727,-20.623683,-2.369532
1276,32,1,0,1,1,2,1,2,-33.177332,15.810200,91.380431,-22.277066,10.900266,-44.947834
1277,32,1,0,1,1,2,1,2,17.423653,-255.251152,7.902810,-30.980470,-48.404122,-135.674829
1278,32,1,0,1,1,2,1,2,-129.179212,107.870991,129.928661,9.256253,138.435465,74.157047


In [11]:
finalData.to_csv('EmotionsData.csv',header=True)

In [ ]:
#Open an SPSS file
df = read_spss(‘PATH_TO_SAV_FILE”)